# Extrae Las preguntas de los PDFs

Este notebook extrae la preguntas de los PDF previamente existentes en una ubicación dada.



## Entorno

### Librerías

In [1]:
from logutils import get_logger
from exam_question_downloader.utils.consts import ROOT_DIR
from pathlib import Path
import pickle
from exam_question_downloader.utils.text_processing import get_text_docs_from_dir, extract_questions_from_doc, classify_questions_by_subject
from exam_question_downloader.utils.text_processing import get_text_docs_from_file, get_questions_not_classified
from exam_question_downloader.utils.text_formating import format_question, format_questions

In [2]:
# SOLO DURANTE EL DESARROLLO; Recarga el código fuente de las librerías en cada ejecucion de celda
%load_ext autoreload
%autoreload 2

In [3]:
logger = get_logger("YEA")

### Variables

In [4]:
# Ubicacion de los PDF a procesar
INPUT_PATH=ROOT_DIR.joinpath("..").joinpath("material-opos").joinpath("PDFsExamenes").joinpath("2024-04-ExamenA1Multimedia")

print(f"Directorio de los exámenes a analizar: {INPUT_PATH}")

Directorio de los exámenes a analizar: /home/tux/Python-Linux/extractor_preguntas_examenes/../material-opos/PDFsExamenes/2024-04-ExamenA1Multimedia


In [5]:
# Fichero de salida de los PDF
OUTPUT_PATH=ROOT_DIR.joinpath("..").joinpath("material-opos").joinpath("CSVsExamenes").joinpath("2024-04-ExamenA1Multimedia")
OUTPUT_PATH.mkdir(exist_ok=True)

print(f"Los CSV se guardarán en: {OUTPUT_PATH}")

Los CSV se guardarán en: /home/tux/Python-Linux/extractor_preguntas_examenes/../material-opos/CSVsExamenes/2024-04-ExamenA1Multimedia


## Leer PDFs

In [6]:
# Buscar archivos PDF en el directorio dado de forma recursiva
archivos_pdf = list(INPUT_PATH.rglob('*.pdf'))

# Mostrar los archivos PDF encontrados
for archivo in archivos_pdf:
    print(archivo)

/home/tux/Python-Linux/extractor_preguntas_examenes/../material-opos/PDFsExamenes/2024-04-ExamenA1Multimedia/Ejercicio Multimedia.pdf
/home/tux/Python-Linux/extractor_preguntas_examenes/../material-opos/PDFsExamenes/2024-04-ExamenA1Multimedia/Plantilla de corrección MULTIMEDIA.pdf


In [7]:
# Cargar en un pickle el texto de todos los PDF
textdocs = []

for archivo in archivos_pdf:
    text_doc = get_text_docs_from_file(archivo)
    textdocs.append((archivo.name, text_doc))

textdocs

[2024-06-24 13:27:08,798] | INFO | text_processing.py:get_text_docs_from_file:105 | File /home/tux/Python-Linux/extractor_preguntas_examenes/../material-opos/PDFsExamenes/2024-04-ExamenA1Multimedia/Ejercicio Multimedia.pdf
[2024-06-24 13:27:08,879] | INFO | text_processing.py:get_text_docs_from_file:105 | File /home/tux/Python-Linux/extractor_preguntas_examenes/../material-opos/PDFsExamenes/2024-04-ExamenA1Multimedia/Plantilla de corrección MULTIMEDIA.pdf


[('Ejercicio Multimedia.pdf',
  ' \nPrimer ejercicio de las pruebas selectivas de acceso al grupo A, \nsubgrupo A1, sector administración especial, categoría de \ntécnico o técnica superiores de multimedia en el Área de \nTecnologías de la Información y las Comunicaciones por el \nsistema de concurso-oposición, correspondiente a la oferta de \nempleo público de 2020. (Código 2023/P/FC/C/2) \n \nPágina 1 de 18 \n \n1) ¿Cuál es la forma política del Estado español según la Constitución? \nA) Monarquía Democrática \nB) Monarquía Parlamentaria \nC) Monarquía Autonómica \nD) Monarquía Constitucional \n2) ¿Cómo se garantiza la representación territorial en Les Corts Valencianes? \nA) A través de un sistema proporcional basado en la población de cada provincia \nB) Mediante un número igual de representantes por cada comarca \nC) Por la elección directa de un representante en cada demarcación electoral \nD) A través de un sistema proporcional basado en la población de cada municipio \n3) ¿Qué 

In [8]:
# Detectar las preguntas y guardarlos en otro picke
questiondocs = []
for name, doc in textdocs:
    questiondocs.append((name, extract_questions_from_doc(doc)))

questiondocs

[2024-06-24 13:27:08,940] | INFO | text_processing.py:extract_questions_from_doc:90 | Question: 1) ¿Cuál es la forma política del Estado español según la Constitución? 
[2024-06-24 13:27:08,942] | INFO | text_processing.py:extract_questions_from_doc:83 | A: A) Monarquía Democrática 
[2024-06-24 13:27:08,943] | INFO | text_processing.py:extract_questions_from_doc:76 | B: B) Monarquía Parlamentaria 
[2024-06-24 13:27:08,945] | INFO | text_processing.py:extract_questions_from_doc:69 | C: C) Monarquía Autonómica 
[2024-06-24 13:27:08,946] | INFO | text_processing.py:extract_questions_from_doc:62 | D: D) Monarquía Constitucional 
[2024-06-24 13:27:08,948] | INFO | text_processing.py:extract_questions_from_doc:40 | Adding question:
Question:1) ¿Cuál es la forma política del Estado español según la Constitución? 
a:A) Monarquía Democrática 
b:B) Monarquía Parlamentaria 
c:C) Monarquía Autonómica 
d:D) Monarquía Constitucional 
[2024-06-24 13:27:08,950] | INFO | text_processing.py:extract_ques

[2024-06-24 13:27:09,022] | INFO | text_processing.py:extract_questions_from_doc:51 | Question: 9) ¿Cuál de los siguientes NO es un Learning Management System (LMS)? 
[2024-06-24 13:27:09,023] | INFO | text_processing.py:extract_questions_from_doc:83 | A: A) Moodle 
[2024-06-24 13:27:09,024] | INFO | text_processing.py:extract_questions_from_doc:76 | B: B) Drupal 
[2024-06-24 13:27:09,025] | INFO | text_processing.py:extract_questions_from_doc:69 | C: C) Canvas 
[2024-06-24 13:27:09,026] | INFO | text_processing.py:extract_questions_from_doc:62 | D: D) Blackboard 
[2024-06-24 13:27:09,027] | INFO | text_processing.py:extract_questions_from_doc:40 | Adding question:
Question:9) ¿Cuál de los siguientes NO es un Learning Management System (LMS)? 
a:A) Moodle 
b:B) Drupal 
c:C) Canvas 
d:D) Blackboard 
[2024-06-24 13:27:09,029] | INFO | text_processing.py:extract_questions_from_doc:51 | Question: 10) ¿En qué opción del menú lateral izquierdo de la implementación de Sakai en 
[2024-06-24 13

[('Ejercicio Multimedia.pdf',
  [{'number': '1',
    'question': '¿Cuál es la forma política del Estado español según la Constitución?',
    'a': 'Monarquía Democrática',
    'b': 'Monarquía Parlamentaria',
    'c': 'Monarquía Autonómica',
    'd': 'Monarquía Constitucional'},
   {'number': '2',
    'question': '¿Cómo se garantiza la representación territorial en Les Corts Valencianes?',
    'a': 'A través de un sistema proporcional basado en la población de cada provincia',
    'b': 'Mediante un número igual de representantes por cada comarca',
    'c': 'Por la elección directa de un representante en cada demarcación electoral',
    'd': 'A través de un sistema proporcional basado en la población de cada municipio'},
   {'number': '3',
    'question': '¿Qué se garantiza en el ámbito del empleo privado y público según la Ley  Orgánica 3/2007, para la igualdad efectiva de mujeres y hombres, de 22 de  marzo?',
    'a': 'Igualdad de salarios entre hombres y mujeres',
    'b': 'Igualdad de

In [9]:
# Estadísticas de las preguntas reconocidas y leídas;

totalpreguntas=0
for doc, preguntas in questiondocs:
    numero_preguntas = len(preguntas)
    totalpreguntas=totalpreguntas+numero_preguntas
    print(f"El archivo '{doc}' tiene {numero_preguntas} preguntas.")

print(f"En total se han leído {totalpreguntas} preguntas.")

El archivo 'Ejercicio Multimedia.pdf' tiene 84 preguntas.
El archivo 'Plantilla de corrección MULTIMEDIA.pdf' tiene 0 preguntas.
En total se han leído 84 preguntas.


In [10]:
# Leemos las preguntas y las clasificamos por temas
df = classify_questions_by_subject(questiondocs)
df

,number,question,a,b,c,d,doc,subject
0,1,¿Cuál es la forma política del Estado español ...,Monarquía Democrática,Monarquía Parlamentaria,Monarquía Autonómica,Monarquía Constitucional,Ejercicio Multimedia.pdf,Constitución Española
1,2,¿Cómo se garantiza la representación territori...,A través de un sistema proporcional basado en ...,Mediante un número igual de representantes por...,Por la elección directa de un representante en...,A través de un sistema proporcional basado en ...,Ejercicio Multimedia.pdf,Corts Valencianes
2,3,¿Qué se garantiza en el ámbito del empleo priv...,Igualdad de salarios entre hombres y mujeres,Igualdad de responsabilidades familiares entre...,Igualdad de trato y de oportunidades en el acc...,Igualdad de representación política entre homb...,Ejercicio Multimedia.pdf,Ley 3/2007
3,4,"Según el objeto de la Ley Orgánica 1/2004, ¿cu...",La violencia física sobre la mujer,La violencia psicológica sobre la mujer,La violencia económica sobre la mujer,La violencia sexual sobre la mujer,Ejercicio Multimedia.pdf,Ley 1/2004
4,5,"Según la Ley de Función Pública Valenciana, ¿c...",Servicio activo,Servicios especiales,Excedencia voluntaria por interés particular,Todas las anteriores conllevan derecho a la re...,Ejercicio Multimedia.pdf,Ley FPV
...,...,...,...,...,...,...,...,...
78,79,¿Cuántas capas tiene la pila TCP/IP estándar?,2,3,4,7 Primer ejercicio de las pruebas selectivas ...,Ejercicio Multimedia.pdf,Redes
79,80,¿Cuál de los siguientes no es un Sistema Opera...,oracle linux,ubuntu,redhat,todos lo son,Ejercicio Multimedia.pdf,Sistemas - Linux
80,81,¿Cuál de estas tecnologías NO se usa principal...,OpenStack,Docker Swarm,Kubernetes,Amazon ECS,Ejercicio Multimedia.pdf,Sistemas - Contenedores
82,83,Señala cuál de los siguientes protocolos de In...,ICMP,TCP,IP,FTP,Ejercicio Multimedia.pdf,Redes


In [11]:
# Estadísticas de la clasificación de preguntas
print (f"De un total de {totalpreguntas} se han clasificado {df.shape[0]} preguntas.")

conteo = df['subject'].value_counts()
print(conteo)



De un total de 84 se han clasificado 75 preguntas.
subject
Open edX                              10
Sakai                                  7
Sistemas - Linux                       6
LMS                                    5
Estatutos UPV                          5
Polimedia y Opencast                   4
Sistemas de audio y vídeo              4
Redes                                  3
IA y realidad virtual                  3
HTML, CSS o Javascript                 3
Poliformat                             3
MOOC                                   3
Redes - IPv6                           2
Sistemas - Contenedores                2
Ley FPV                                2
Creative Commons                       2
Riunet                                 2
Sistemas - Windows 10 y Windows 11     2
Prevención de riesgos                  1
Ley 3/2007                             1
Ley 1/2004                             1
Constitución Española                  1
Corts Valencianes                      

## Preguntas no clasificadas en temas

In [12]:
# Las preguntas que no se han clasificado
df2 = get_questions_not_classified(questiondocs)

df2

,number,question,a,b,c,d,doc,subject
0,30,¿Cuánto texto se debe incluir en una transpare...,El mínimo imprescindible,Dos líneas,Tres líneas,5 líneas,Ejercicio Multimedia.pdf,None
1,33,En el ámbito de la UPV un objeto de aprendizaj...,Debe tener propósito pedagógico,Debe tener propósito pedagógico y ser indivisi...,"Debe tener propósito pedagógico, ser indivisib...","Debe tener propósito pedagógico, ser indivisib...",Ejercicio Multimedia.pdf,None
2,40,El motor de diseño 3D Unreal:,Es de código abierto,"No es de código abierto, pero es gratuito",Es gratuito para empresas y particulares que g...,"Es siempre de pago, con una cuota anual por es...",Ejercicio Multimedia.pdf,None
4,63,¿Cuál de las siguientes es una función específ...,Incrementar la seguridad del servidor mediante...,Distribuir el tráfico entrante entre varios se...,Comprimir los datos antes de enviarlos a los c...,Realizar análisis detallados del tráfico para ...,Ejercicio Multimedia.pdf,None
6,67,¿Cuál es el uso correcto del método replace() ...,"texto.replace('perro', 'gato')","replace.texto('perro', 'gato')","replace(texto,'perro', 'gato')",texto('perro').replace('gato'),Ejercicio Multimedia.pdf,None
7,69,¿Qué consulta SQL combina información de las t...,SELECT productos.nombre FROM productos LEFT JO...,SELECT productos.nombre FROM productos INNER J...,"SELECT productos.nombre FROM productos, catego...",SELECT productos.nombre FROM productos LEFT JO...,Ejercicio Multimedia.pdf,None
8,70,¿Cuál de las siguientes es un entorno de desar...,Bootstrap,Ansible,Django,NodeJS,Ejercicio Multimedia.pdf,None
10,73,¿Cuál es una herramienta de transcripción de a...,Rask,Heygen,Elevenlabs,Whisper,Ejercicio Multimedia.pdf,None
12,82,¿Qué objeto de base de datos es necesario crea...,listener,dblink,conector,ssh,Ejercicio Multimedia.pdf,None


In [13]:
print(format_question(df2.iloc[0]))

¿Cuánto texto se debe incluir en una transparencia de una presentación para un  vídeo educativo?
  a) El mínimo imprescindible
  b) Dos líneas
  c) Tres líneas
  d) 5 líneas


In [14]:
print(format_questions(df2))


¿Cuánto texto se debe incluir en una transparencia de una presentación para un  vídeo educativo?
  a) El mínimo imprescindible
  b) Dos líneas
  c) Tres líneas
  d) 5 líneas

En el ámbito de la UPV un objeto de aprendizaje en vídeo deberá tener las  siguientes características:
  a) Debe tener propósito pedagógico
  b) Debe tener propósito pedagógico y ser indivisible e independiente
  c) Debe tener propósito pedagógico, ser indivisible e independiente y ser reutilizable
  d) Debe tener propósito pedagógico, ser indivisible e independiente, ser reutilizable  y tener menos de 2 años de
antigüedad para asegurar su vigencia

El motor de diseño 3D Unreal:
  a) Es de código abierto
  b) No es de código abierto, pero es gratuito
  c) Es gratuito para empresas y particulares que generen menos de un millón de euros  de ingresos brutos anuales
  d) Es siempre de pago, con una cuota anual por estación de desarrollo

¿Cuál de las siguientes es una función específica de Nginx?
  a) Incrementar la s

## Exportación de las preguntas clasificadas al directorio de salida


In [15]:
from datetime import datetime

now = datetime.now().strftime("%Y-%m-%d")
filename=now+"-preguntas.csv"
filepath= OUTPUT_PATH.joinpath(filename)

print(f"Nombre del fichero de salida: {filename}")
print(f"Ruta del fichero de salida: {filepath}")

Nombre del fichero de salida: 2024-06-24-preguntas.csv
Ruta del fichero de salida: /home/tux/Python-Linux/extractor_preguntas_examenes/../material-opos/CSVsExamenes/2024-04-ExamenA1Multimedia/2024-06-24-preguntas.csv


In [16]:
# Exportación
df.to_csv(filepath, index=False)

In [17]:
# Guardamos un informe de la exportación

# Nombre del fichero
filename=now+"-informe.txt"
filepath= OUTPUT_PATH.joinpath(filename)

# Abrir el fichero en modo escritura
with open(filepath, 'w') as file:

    totalpreguntas=0
    for doc, preguntas in questiondocs:
        numero_preguntas = len(preguntas)
        totalpreguntas=totalpreguntas+numero_preguntas
        print(f"El archivo '{doc}' tiene {numero_preguntas} preguntas.")
        file.write(f"El archivo '{doc}' tiene {numero_preguntas} preguntas.\n")

    print(f"En total se han leído {totalpreguntas} preguntas.")
    file.write(f"En total se han leído {totalpreguntas} preguntas.\n")

    print(f"Del total de {totalpreguntas} se han clasificado {df.shape[0]} preguntas.")
    file.write(f"Del total de {totalpreguntas} se han clasificado {df.shape[0]} preguntas.\n")

El archivo 'Ejercicio Multimedia.pdf' tiene 84 preguntas.
El archivo 'Plantilla de corrección MULTIMEDIA.pdf' tiene 0 preguntas.
En total se han leído 84 preguntas.
Del total de 84 se han clasificado 75 preguntas.
